<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/single_side_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#connect drive with colab notebook
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam,RMSprop
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [0]:
!mkdir data

In [0]:
#size of images to feed in neural network
image_shape = (448,448,3) 
#data augmentation

datagen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               vertical_flip=True,
                               fill_mode='nearest',# Fill in missing pixels with the nearest filled value
                               validation_split=0.2#split data to train and test
                              )

In [8]:
ls

Cataract/  Diabetes/  Glaucoma/  Hypertension/  Myopia/  Normal/  Other/


In [6]:
#load the training data
train_generator = datagen.flow_from_directory('/content/drive/My Drive/ghina/thesis/new_data/',
    target_size=image_shape[0:2],
    batch_size=100,
    class_mode='categorical',
    subset='training')

Found 2802 images belonging to 7 classes.


In [7]:
test_generator = datagen.flow_from_directory('/content/drive/My Drive/ghina/thesis/new_data/',
    target_size=image_shape[0:2],
    batch_size=100,
    class_mode='categorical',
    subset='validation')

Found 698 images belonging to 7 classes.


In [8]:
from keras.applications.resnet50 import ResNet50

model =ResNet50(weights = "imagenet", include_top=False, input_shape = image_shape)
for layer in model.layers:
    layer.trainable = False
x = model.output
x=  keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.2)(x)
predictions = Dense(7, activation="softmax")(x)


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [0]:
model = Model(inputs=model.input, outputs=predictions)


In [0]:
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [0]:
from keras.optimizers import Adam,RMSprop
opt = RMSprop(lr=1e-3,decay=0.2)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[macro_f1])

In [0]:
#create class weights for unbalance dataset
from sklearn.utils import class_weight
y_train=train_generator.classes
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
#train model
results = model.fit_generator(train_generator,epochs=40,
                              steps_per_epoch=30,
                              validation_data=test_generator,
                             validation_steps=30,
                             class_weight=class_weights)

Epoch 1/40
30/30 [==============================] - 626s 21s/step - loss: 1.7319 - macro_f1: 0.2594 - val_loss: 3.5140 - val_macro_f1: 0.0084
Epoch 2/40
30/30 [==============================] - 521s 17s/step - loss: 1.3269 - macro_f1: 0.2902 - val_loss: 5.0052 - val_macro_f1: 0.0109
Epoch 3/40
30/30 [==============================] - 551s 18s/step - loss: 1.2278 - macro_f1: 0.2957 - val_loss: 5.5756 - val_macro_f1: 0.0105
Epoch 4/40
21/30 [====================>.........] - ETA: 40s - loss: 1.3018 - macro_f1: 0.2865